# Breast Cancer Prediction

In [ ]:
!pip install ucimlrepo gradio

<h4> Importing all the necessary libraries </h4>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

# Necessary imports for GUI done by using gradio
from ucimlrepo import fetch_ucirepo
import gradio as gr

<h4>Importing the data from data.csv into pandas DataFrame</h4>

In [ ]:
data = pd.read_csv('/content/data.csv')

<h4>Data Preprocessing</h4>

In [ ]:
data = data.drop(data.columns[0],axis=1)
data = data.drop(data.columns[31],axis=1)
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})
x = data.loc[:, data.columns != 'diagnosis'].values
y = data.loc[:, 'diagnosis'].values

<h4>Scaling the Data using StandardScalar library</h4>

In [ ]:
sc_X = StandardScaler()
X = sc_X.fit_transform(x)

## Hyperparameter Tuning

In [ ]:
# Perform Grid Search for SVM hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(svm.SVC(), param_grid, scoring='accuracy', cv=5)
grid_search.fit(X, y)
best_svm = grid_search.best_estimator_
print("Best Hyperparameters for SVM:", grid_search.best_params_)
svm_cl = best_svm

Best Hyperparameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


<h4>Initialising all the classifiers</h4>

In [ ]:
knn_cl = KNeighborsClassifier(n_neighbors=23, p=2, metric='euclidean')
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
clf = RandomForestClassifier(max_depth=2, random_state=0)
svm_cl = svm.SVC(kernel='rbf', gamma='auto', C=1)
nb_cl = GaussianNB()

<h4>Cross Validation (5 folds)</h4>

In [ ]:
scoring = ['accuracy','precision','f1','recall']
scores1 = cross_validate(model, X, y, scoring=scoring)
scores2 = cross_validate(clf, X, y, scoring=scoring)
scores3 = cross_validate(knn_cl, X, y, scoring=scoring)
scores4 = cross_validate(svm_cl, X, y, scoring=scoring)
scores5 = cross_validate(nb_cl, X, y, scoring=scoring)

<h4>Fitting the dataset to the model</h4>

In [ ]:
model.fit(X, y)
clf.fit(X,y)
knn_cl.fit(X, y)
svm_cl.fit(X, y)
nb_cl.fit(X, y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:40:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GaussianNB()

<h4>Calculation of evaluation metrics</h4>

In [ ]:
label_arr = ['XGB','RF','KNN','SVM','NB']
acc_arr = [round(scores1['test_accuracy'].mean(),3),round(scores2['test_accuracy'].mean(),3),round(scores3['test_accuracy'].mean(),3),
           round(scores4['test_accuracy'].mean(),3),round(scores5['test_accuracy'].mean(),3)]
prec_arr = [round(scores1['test_precision'].mean(),3),round(scores2['test_precision'].mean(),3),round(scores3['test_precision'].mean(),3),
           round(scores4['test_precision'].mean(),3),round(scores5['test_precision'].mean(),3)]
f1_arr = [round(scores1['test_f1'].mean(),3),round(scores2['test_f1'].mean(),3),round(scores3['test_f1'].mean(),3),
           round(scores4['test_f1'].mean(),3),round(scores5['test_f1'].mean(),3)]
rec_arr = [round(scores1['test_recall'].mean(),3),round(scores2['test_recall'].mean(),3),round(scores3['test_recall'].mean(),3),
           round(scores4['test_recall'].mean(),3),round(scores5['test_recall'].mean(),3)]

<h4>Printing accuracy values for each model</h4>

In [ ]:
print(scores1['test_accuracy'].mean())
print("\n")
print(scores2['test_accuracy'].mean())
print("\n")
print(scores3['test_accuracy'].mean())
print("\n")
print(scores4['test_accuracy'].mean())
print("\n")
print(scores5['test_accuracy'].mean())
print("\n")

0.9701288619779538


0.9490607048594939


0.9543393882937432


0.9736376339077782


0.927930445582984




# GUI

<h4>DECLARING FUNCTION FOR GRAPH</h4>

In [ ]:
def make_plot(plot_type):
    labels = list(label_arr)
    if plot_type == "Accuracy":
        values = list(acc_arr)

        plt.bar(labels, values, color ='yellow',
                width = 0.7)
        for i in range(len(labels)):
            plt.text(i,values[i]/2,values[i],ha = 'center')
        plt.xlabel("MODEL")
        plt.ylabel("Accuracy")
        plt.title("Accuracy of different models")
        plt.show()
    elif plot_type == "Precision":
        values = list(prec_arr)
        plt.bar(labels, values, color ='yellow',
                width = 0.7)
        for i in range(len(labels)):
            plt.text(i,values[i]/2,values[i],ha = 'center')
        plt.xlabel("MODEL")
        plt.ylabel("Precision")
        plt.title("Precision of different models")
        plt.show()
    elif plot_type == "F1":
        values = list(f1_arr)
        plt.bar(labels, values, color ='yellow',
                width = 0.7)
        for i in range(len(labels)):
            plt.text(i,values[i]/2,values[i],ha = 'center')
        plt.xlabel("MODEL")
        plt.ylabel("F1 score")
        plt.title("F1 score of different models")
        plt.show()
    elif plot_type == "Recall":
        values = list(rec_arr)
        plt.bar(labels, values, color ='yellow',
                width = 0.7)
        for i in range(len(labels)):
            plt.text(i,values[i]/2,values[i],ha = 'center')
        plt.xlabel("MODEL")
        plt.ylabel("F1 score")
        plt.title("F1 score of different models")
        plt.show()
    return plt

<h4>DECLARING FUNCTION FOR PREDICTION</h4>

In [ ]:
def calc_xgb(radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,
        symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,
         concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,
         compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst):
    x_arr = [radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,
        symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,
         concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,
         compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst]
    pred = svm_cl.predict(np.array(x_arr).reshape(1,30))
    if pred[0] == 1:
        return 'Malignant Tumor'
    else:
        return 'Benign Tumor'

<h4>PREDICTION AND GRAPH TAB</h4>

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("CHECK IF BREAST TUMOR IS MALIGNANT OR BENIGN")
    with gr.Tab("SVM"):
        with gr.Row():
            radius_mean=gr.Number(label="Radius Mean(μm)" )
            radius_se=gr.Number(label="Radius SE (μm)" )
            radius_worst=gr.Number(label="Radius Worst (μm)" )

        with gr.Row():
            texture_mean=gr.Number(label="Texture Mean (nm) " )
            texture_se=gr.Number(label="Texture SE (nm) " )
            texture_worst=gr.Number(label="Texture Worst (nm) " )

        with gr.Row():
            perimeter_mean=gr.Number(label="Perimeter Mean (μm)" )
            perimeter_se=gr.Number(label="Perimeter SE (μm)" )
            perimeter_worst=gr.Number(label="Perimeter Worst (μm)" )
        with gr.Row():
            area_mean=gr.Number(label="Area Mean (sq.μm)" )
            area_se=gr.Number(label="Area SE (sq.μm)" )
            area_worst=gr.Number(label="Area Worst (sq.μm)" )

        with gr.Row():
            smoothness_mean=gr.Number(label="Smoothness Mean (nm)" )
            smoothness_se=gr.Number(label="Smoothness SE (nm)")
            smoothness_worst=gr.Number(label="Smoothness Worst (nm)")
        with gr.Row():
            compactness_mean=gr.Number(label="Compactness Mean (nm)")
            compactness_se=gr.Number(label="Compactness SE (nm)")
            compactness_worst=gr.Number(label="Compactness Worst (nm)")

        with gr.Row():
            concavity_mean=gr.Number(label="Concavity Mean " )
            concavity_se=gr.Number(label="Concavity SE " )
            concavity_worst=gr.Number(label="Concavity Worst " )
        with gr.Row():
            concave_points_mean=gr.Number(label="Concave Points Mean " )
            concave_points_se=gr.Number(label="Concave Points SE " )
            concave_points_worst=gr.Number(label="Concave Points Worst " )

        with gr.Row():
            symmetry_mean=gr.Number(label="Symmetry Mean" )
            symmetry_se=gr.Number(label="Symmetry SE" )
            symmetry_worst=gr.Number(label="Symmetry Worst" )
        with gr.Row():
            fractal_dimension_mean=gr.Number(label="Fractal Dimension Mean" )
            fractal_dimension_se=gr.Number(label="Fractal Dimension SE" )
            fractal_dimension_worst=gr.Number(label="Fractal Dimension Worst" )

        text_output = gr.Textbox(label='Prediction')

        xgb_button = gr.Button("Predict")

    with gr.Tab("Graph"):
        #dim=gr.Number(label="Hmdxx Min", info="Choose between -50 and 50")

        plot = gr.Plot(label="Plot")
        button = gr.Radio(label="Metric",
                      choices=['Accuracy', 'Precision', 'F1','Recall'], value='scatter_plot')
        #plot = gr.Plot(label="Plot")
        button.change(make_plot, inputs=button, outputs=[plot])
        #graph_button.click(make_plot,inputs = [m,y],outputs=[plot])
        #demo.load(make_plot,outputs=[plot])

    xgb_button.click(calc_xgb, inputs=[radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,
        symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,
         concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,
         compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst], outputs=text_output)
    #disp_button.click(r_square,  outputs=[xgb_output,rf_output,svr_output,mlr_output,lr_output,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1774a8ccb93e65594e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
